In [1]:
import pyvoro
import random
import numpy as np
import time
from scipy.spatial import ConvexHull
import sys
sys.path.append("..")
from utils import ccw_check


In [2]:
def generate_voronoi(numVertices):
    points = np.random.rand(numVertices,3)

# Ajustar los límites para el cubo de 1000x1000x1000
    limits = [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]

    # Se puede ajustar el tamaño del bloque según sea necesario
    block_size = 1.0 # Este valor puede variar según tus necesidades

    # Calcular el diagrama de Voronoi
    t0 = time.time()
    voronoi_result = pyvoro.compute_voronoi(points, limits, block_size)
    tf = time.time()
    print('voronoi time = ', (tf-t0))
    vertexs = []
    polyhedrons = []
    minx = 0
    maxx = 0
    miny = 0
    maxy = 0
    minz = 0
    maxz = 0
    for cell in voronoi_result:
        vertexs+=cell['vertices']
        vertices = np.array(cell['vertices'])  # Convierte a array de NumPy para facilitar el manejo
        faces = cell['faces']
        cell_faces = []
        minx = min(minx,vertices[:, 0].min())
        miny = min(miny,vertices[:, 1].min())
        minz = min(minz,vertices[:, 2].min())

        maxx = max(maxx,vertices[:, 0].max())
        maxy = max(maxy,vertices[:, 1].max())
        maxz = max(maxz,vertices[:, 2].max())
        
        for face in faces:
            cell_faces.append(face['vertices'])

        polyhedrons.append([cell['vertices'],cell_faces, cell['volume']])

    # print(minx,maxx,miny,maxy,minz ,maxz)
    return polyhedrons

In [3]:
from math import sqrt
from statistics import mean



def edges(polyhedron):
    edge_list=[]
    for face in polyhedron[1]:
        for i in range(len(face)):
            ei = [face[i%len(face)],face[(i+1)%len(face)]]
            ei.sort()
            if ei not in edge_list:
                edge_list.append(ei)

    return edge_list

def edges_length_ratio(polyhedron):
    length_list=[]
    for face in polyhedron[1]:
        for i in range(len(face)):
            ei = [face[i%len(face)],face[(i+1)%len(face)]]
            ei.sort()
            if ei not in length_list:
                v1 = polyhedron[0][ei[0]]
                v2 = polyhedron[0][ei[1]]
                l = sqrt(((v2[0]-v1[0])**2 + (v2[1]-v1[1])**2 + (v2[2]-v1[2])**2))
                length_list.append(l)

    shortest = min(length_list)
    longest = max(length_list)
    return shortest/longest

def n_faces(polyhedron):
    return len(polyhedron[1])

# area:
def face_area(face_vertex):
    area = [0,0,0]
    v0 = face_vertex[0]
    for i in range(1,len(face_vertex)-1):
        v1 = face_vertex[i]
        v2 = face_vertex[i+1]
        A = np.array([v1[0]-v0[0], v1[1]-v0[1], v1[2]-v0[2]])
        B = np.array([v2[0]-v0[0], v2[1]-v0[1], v2[2]-v0[2]])
        local_area = np.cross(A,B)
        area[0]+= local_area[0]
        area[1]+= local_area[1]
        area[2]+= local_area[2]
    return 0.5 * sqrt((area[0]**2 + area[1]**2) + area[2]**2)


def polyhedron_area_ratio(polyhedron):
    area = 0
    nodes = np.array(polyhedron[0])
    for face in polyhedron[1]:
        face_vertex = nodes[face]
        area += face_area(face_vertex)
    cvhull = ConvexHull(nodes)
    cvHull_area = cvhull.area
    ratio = area / cvHull_area
    return ratio

def is_convex(polyhedron):
    nodes = polyhedron[0]
    cvhull = ConvexHull(nodes)
    return set(cvhull.vertices) == set(range(len(nodes)))


# def write_polyhedron_file(filename,polyhedrons):
#     with open(filename+'.txt', 'w') as fh:
#         # polys = self.polyhedral_mesh
#         p = len(polyhedrons)
#         fh.write(str(p)+'\n')
#         c = 0
#         for polyhedron in polyhedrons:
#             list_face = polyhedron[1]
#             nodes = polyhedron[0]
#             # for tetra in polyhedron.tetras:
#             #     vertex = [self.mesh.tetra_list[tetra].v1, self.mesh.tetra_list[tetra].v2,self.mesh.tetra_list[tetra].v3,self.mesh.tetra_list[tetra].v4]
#             #     for v in vertex:
#             #         if v not in nodes :
#             #             nodes.append(v) 
#             fh.write("%d %d\n" % (len(nodes), len(list_face)))
#             for node in nodes:
#                 # v = self.mesh.node_list[node]
#                 fh.write("%f %f %f\n" % (node[0], node[1], node[2]))
#             nodes = np.array(polyhedron[0])
#             for f in list_face:
#                 # print(list_face)
#                 face_vertex = nodes[face]
#                 # t = self.mesh.face_list[f].neighs[0] if (self.mesh.face_list[f].neighs[0] in polyhedron.tetras) else self.mesh.face_list[f].neighs[1]
#                 if ccw_check(self.mesh.face_list[f], self.mesh.tetra_list[t],self.mesh.node_list):
#                     v1 = nodes.index(self.mesh.face_list[f].v1)
#                     v2 = nodes.index(self.mesh.face_list[f].v2)
#                     v3 = nodes.index(self.mesh.face_list[f].v3)
#                 else:
#                     v1 = nodes.index(self.mesh.face_list[f].v1)
#                     v2 = nodes.index(self.mesh.face_list[f].v3)
#                     v3 = nodes.index(self.mesh.face_list[f].v2)
#                 fh.write("3 %d %d %d # %d %d\n" % (v1, v2, v3,f,c))
#             c+=1


def analize_voronoi(numVertices):
    polyhedrons = generate_voronoi(numVertices)

    edge_ratios = []
    n_faces_list = []
    areas = []
    volumen = []
    convexs_polys = 0
    for polyhedron in polyhedrons:
        # print('vertices: ', polyhedron[0])
        # print('faces:', polyhedron[1])
        # print(edges(polyhedron))
        edge_ratio = edges_length_ratio(polyhedron)
        edge_ratios.append(edge_ratio)
        n_face = n_faces(polyhedron)
        n_faces_list.append(n_face)
        area = polyhedron_area_ratio(polyhedron)
        areas.append(area)
        volumen.append(polyhedron[2])
        if is_convex(polyhedron):
            convexs_polys+=1
    print('Cantidad de Poliedros; ', len(polyhedrons))
    print('Cantidad de Poliedros convexos; ', convexs_polys)
    print('Promedio Edge ratio: ', mean(edge_ratios))
    print('Minimum Edge ratio: ', min(edge_ratios))
    print('Maximum Edge ratio: ', max(edge_ratios))

    print('Promedio Numero de Caras: ', mean(n_faces_list))
    print('Minimum Numero de Caras: ', min(n_faces_list))
    print('Maximum Numero de Caras: ', max(n_faces_list))

    print('Promedio Area Ratio: ', mean(areas))
    print('Minimum Area Ratio: ', min(areas))
    print('Maximum Area Ratio: ', max(areas))

    print('Promedio Volume: ', mean(volumen))
    print('Minimum Volume: ', min(volumen))
    print('Maximum Volume: ', max(volumen))


In [8]:
analize_voronoi(100)


voronoi time =  0.01393890380859375
-2.7755575615628914e-17 1.0 -2.7755575615628914e-17 1.0 -2.7755575615628914e-17 1.0
Cantidad de Poliedros;  100
Cantidad de Poliedros convexos;  100
Promedio Edge ratio:  0.034926539827858774
Minimum Edge ratio:  0.0008302878675099121
Maximum Edge ratio:  0.17887856580275352
Promedio Numero de Caras:  12.02
Minimum Numero de Caras:  6
Maximum Numero de Caras:  20
Promedio Area Ratio:  1.0
Minimum Area Ratio:  0.9999999999999996
Maximum Area Ratio:  1.0000000000000004
Promedio Volume:  0.009999999999999998
Minimum Volume:  0.0006473167084012498
Maximum Volume:  0.0228645875168903


In [9]:
analize_voronoi(1000)

voronoi time =  0.20186400413513184
-2.7755575615628914e-17 1.0 -4.163336342344337e-17 1.0 -1.3877787807814457e-17 1.0
Cantidad de Poliedros;  1000
Cantidad de Poliedros convexos;  1000
Promedio Edge ratio:  0.02431230916246761
Minimum Edge ratio:  1.5856760275274876e-07
Maximum Edge ratio:  0.4371730420462092
Promedio Numero de Caras:  13.777
Minimum Numero de Caras:  5
Maximum Numero de Caras:  27
Promedio Area Ratio:  1.0
Minimum Area Ratio:  0.9999999999999991
Maximum Area Ratio:  1.0000000000000007
Promedio Volume:  0.001
Minimum Volume:  9.735959982582328e-05
Maximum Volume:  0.003192829438560447


In [10]:
analize_voronoi(10000)

voronoi time =  7.823276996612549
-1.3877787807814457e-17 1.0 -1.3877787807814457e-17 1.0 -1.3877787807814457e-17 1.0
Cantidad de Poliedros;  10000
Cantidad de Poliedros convexos;  10000
Promedio Edge ratio:  0.019221147004610615
Minimum Edge ratio:  8.645298542791992e-06
Maximum Edge ratio:  0.6390256498824958
Promedio Numero de Caras:  14.7443
Minimum Numero de Caras:  5
Maximum Numero de Caras:  29
Promedio Area Ratio:  1.0
Minimum Area Ratio:  0.999999999999999
Maximum Area Ratio:  1.0000000000000009
Promedio Volume:  9.999999999999999e-05
Minimum Volume:  9.463412421270387e-06
Maximum Volume:  0.00045320028667753545


In [4]:
analize_voronoi(50000)

voronoi time =  157.92708325386047
-1.3877787807814457e-17 1.0 -1.3877787807814457e-17 1.0 -1.0408340855860843e-17 1.0


: 